In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import  Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.common.by import  By 
from selenium.webdriver.support import  expected_conditions as EC 

import pandas as pd
import numpy as np
import time
import datetime

In [2]:
#建立数据库
columns=['时间','发布日期','作者名','作者类型','粉丝数','投稿数','视频名','分区','子分区','视频类型','自制or转载','标签','视频长度','播放量','点赞数','投币数','收藏数','转发数','评论数','弹幕数','有无分集']
channel_shenghuo_shougong=pd.DataFrame(columns=columns)
channel_shenghuo_shougong

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集


In [3]:
driver=webdriver.Chrome()
driver.get('https://passport.bilibili.com/login') 

In [4]:
#登录
driver.find_element_by_xpath('//*[@id="login-username"]').send_keys('15717506191')
driver.find_element_by_xpath('//*[@id="login-passwd"]').send_keys('xy000000')
driver.find_element_by_xpath('//*[@id="geetest-wrap"]/div/div[5]/a[1]').click()
#需要手动过验证码

In [5]:
driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[9]/div/a/span').click()  #选择“生活”频道

In [5]:
#获得分区名
channel=driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[9]/div/a/span').text
channel=channel.strip('0123456789+')
channel

'生活'

In [6]:
driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[5]/a').click()  #选择“手工”子频道
sub_channel=driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[5]/a').text
sub_channel

'手工'

In [7]:
driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[1]/ul/a[2]/li').click() #让视频按热度排序

In [11]:
print(datetime.datetime.now())

for month in range(7,11): 
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div/span').click()
    period=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).text
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).click()
    WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]/div/div[2]/div[3]/a'))) 
    for page in range(5):    #爬取每个频道1月到10月每个月播放量前100的视频

        video_loc=1
        while video_loc<21:

            ### 判断作者类型###
            
            
            driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).click()
            driver.switch_to_window(driver.window_handles[-1])
            
            
            author_web_show='No'
            try:
                i=0
                while author_web_show=='No':
                    driver.refresh()
                    i+=1
                    try:
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="n-fs"]')))  #看作者名是否出现
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="navigator"]/div/div[1]/div[1]/a[3]/span[3]')))  #看投稿数是否出现
                        author_web_show='Yes'
                    except:
                        author_web_show='No' 
                        if i>3:  
                            raise Exception  #刷新几次后仍爬不了说明用户已注销

                try:
                    driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m organization-auth"]')
                    author_type='机构认证'

                except:
                    try:
                        driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m personal-auth"]')
                        author_type='个人认证'
                    except:
                        author_type='无'

                fans_num=driver.find_element_by_xpath('//*[@id="n-fs"]').text  #获取作者粉丝数

                ### 获取作者投稿数 ###
                driver.find_element_by_xpath('//*[@class="n-num"]').click()
                WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span')))  
                video_num=driver.find_element_by_xpath('//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span').text

                driver.close()
                driver.switch_to_window(driver.window_handles[-1])


                view_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[1]/span'%video_loc).text #获取播放量
                real_dan_mu_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[2]/span'%video_loc).text #获取弹幕数
                author=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).text #获取作者名
                driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/a'%video_loc).click() #进入视频界面  
                driver.switch_to_window(driver.window_handles[-1]) 

                try:
                    WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))  #若出现视频被删除的情况则退出

                    video_web_show='No'
                    while video_web_show=='No':
                        driver.refresh()

                        try:
                            WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@class="bilibili-player-video-time-total"]'))) #以视频长度出现时间判定网页是否加载完
                            video_web_show='Yes'

                            try:      
                                driver.find_element_by_xpath('//*[@id="multi_page"]')   #测试是否有分集视频
                                multi='是'

                            except:
                                multi='否'

                            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
                            driver.execute_script('window.scrollTo(document.body.scrollHeight,0)') #再调回最上面是因为，视频长度的元素必须在屏幕内才可以被提取，否则为空

                            bv=driver.current_url[33:44] # 获取BV号（BV号在网址中）
                            print('开始BV号为%s视频属性的抓取'%bv)
                            video_name=driver.find_element_by_xpath('//*[@id="viewbox_report"]/h1/span').text #获取视频名
                            
                            ### 获取发布日期 ###
                            release_date=driver.find_element_by_xpath('//*[@id="viewbox_report"]/div/span[3]').text
                            
                            ### 获取视频类型 ###
                            try:
                                video_type=driver.find_element_by_xpath('//*[@class="activity"]').text
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type=video_type+'和合作视频'
                                except:
                                    pass
                            except:
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type='合作视频'
                                except:
                                    video_type='普通视频'


                            ### 获取点赞数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text=='点赞':
                                like_num=0
                            else:
                                like_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text 

                            ### 获取投币数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text=='投币':
                                coin_num=0
                            else:
                                coin_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text

                            ### 获取收藏数 ###             
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text=='收藏':
                                collect_num=0
                            else:
                                collect_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text

                            ### 获取转发数 ###       
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text=='分享':
                                forward_num=0
                            else:
                                forward_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text

                            ### 获取视频长度 ###

                            video_length=driver.find_element_by_xpath('//*[@class="bilibili-player-video-time-total"]').text  

                            ### 判断自制还是转载 ###
                            driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').click()
                            try:
                                driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/div[2]/div[2]').text
                                self_made='自制'
                            except:
                                self_made='转载'
                            driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/i').click()

                            ### 获取标签 ###
                            k=1
                            labels=np.array
                            while True:
                                try:
                                    labels=np.append(labels,driver.find_element_by_xpath('//*[@id="v_tag"]/ul/li[%d]'%k).text)
                                    k+=1
                                except: 
                                    break
                            labels=labels[1:]

                            ### 获取评论数 ###
                            driver.execute_script('window.scrollTo(0, 700)')        
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[2]'))) #保证"评论"二字能加载出来                                                   
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div')))
                            if driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/div/div[4]/div').text=='没有更多信息':    #应对评论区被关闭的情况
                                comment_num='NaN'
                            else:
                                while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                    try:
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div[1]')))
                                        while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                            time.sleep(0.1) #很奇怪，评论数可以定位的一瞬间并不能爬取，会返回空
                                        comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text     
                                    except:
                                        driver.refresh()    #有的时候“评论”二字出现，评论数不出现，页面好像卡住了
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))
                                        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[1]')))

                                comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text

                            channel_shenghuo_shougong.loc[bv]=[period,release_date,author,author_type,fans_num,video_num,video_name,channel,sub_channel,video_type,self_made,labels,video_length,view_num,like_num,coin_num,collect_num,forward_num,comment_num,real_dan_mu_num,multi]

                            print('BV号为%s的视频属性抓取结束'%bv)

                            driver.close()
                            driver.switch_to_window(driver.window_handles[-1])

                        except:
                            video_web_show='No' 
                except:
                    driver.close()
                    driver.switch_to_window(driver.window_handles[-1])
            except:
                driver.close()
                driver.switch_to_window(driver.window_handles[-1])
            
            video_loc+=1


        driver.find_element_by_xpath('//*[@class="nav-btn iconfont icon-arrowdown3"]').click() #点击下一页
        WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]')))  #点击下一页后等页面刷新
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[2]').click()  #回到第1页
    WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]')))
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]').click()  #发现很多月份漏爬了约20个，而且恰好是第一页，所以再点一次第一页
        
        
print(datetime.datetime.now()) 

2020-12-22 07:42:14.228274


<ipython-input-11-f88e7cbd8d2a>:17: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-11-f88e7cbd8d2a>:54: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-11-f88e7cbd8d2a>:61: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1Ya4y1a7DC视频属性的抓取
BV号为1Ya4y1a7DC的视频属性抓取结束


<ipython-input-11-f88e7cbd8d2a>:181: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1LA411Y7mW视频属性的抓取
BV号为1LA411Y7mW的视频属性抓取结束
开始BV号为1sa4y1a74g视频属性的抓取
BV号为1sa4y1a74g的视频属性抓取结束
开始BV号为1fA411a7Ge视频属性的抓取
BV号为1fA411a7Ge的视频属性抓取结束
开始BV号为1Xt4y197et视频属性的抓取
BV号为1Xt4y197et的视频属性抓取结束
开始BV号为1Ba4y1E75G视频属性的抓取
BV号为1Ba4y1E75G的视频属性抓取结束
开始BV号为1GZ4y1u7k6视频属性的抓取
BV号为1GZ4y1u7k6的视频属性抓取结束
开始BV号为1Wi4y1V7aC视频属性的抓取
BV号为1Wi4y1V7aC的视频属性抓取结束
开始BV号为1Gz4y1D7UR视频属性的抓取
BV号为1Gz4y1D7UR的视频属性抓取结束
开始BV号为1AT4y1j7xQ视频属性的抓取
BV号为1AT4y1j7xQ的视频属性抓取结束
开始BV号为1jT4y1E7Ce视频属性的抓取
BV号为1jT4y1E7Ce的视频属性抓取结束
开始BV号为1vD4y1Q7J7视频属性的抓取
BV号为1vD4y1Q7J7的视频属性抓取结束
开始BV号为1Sf4y1X7QE视频属性的抓取
BV号为1Sf4y1X7QE的视频属性抓取结束
开始BV号为1164y1F71H视频属性的抓取
BV号为1164y1F71H的视频属性抓取结束
开始BV号为1HD4y1U7Lc视频属性的抓取
BV号为1HD4y1U7Lc的视频属性抓取结束
开始BV号为1ZZ4y1M7BH视频属性的抓取
BV号为1ZZ4y1M7BH的视频属性抓取结束
开始BV号为1di4y1G7hb视频属性的抓取
BV号为1di4y1G7hb的视频属性抓取结束
开始BV号为1hp4y1S7Mr视频属性的抓取
BV号为1hp4y1S7Mr的视频属性抓取结束
开始BV号为1sZ4y1u7kQ视频属性的抓取
BV号为1sZ4y1u7kQ的视频属性抓取结束
开始BV号为18z411v76e视频属性的抓取
BV号为18z411v76e的视频属性抓取结束
开始BV号为1BZ4y1u71n视频属性的抓取
BV号为1BZ4y1u71n的视频属性抓取结束
开始BV号为1iz411v73U视频属性的抓取
BV号为1iz411v73U的视

<ipython-input-11-f88e7cbd8d2a>:187: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1GK411T7gg视频属性的抓取
BV号为1GK411T7gg的视频属性抓取结束
开始BV号为1YK4y1e79L视频属性的抓取
BV号为1YK4y1e79L的视频属性抓取结束
开始BV号为1jK411T7pe视频属性的抓取
BV号为1jK411T7pe的视频属性抓取结束
开始BV号为1RT4y157vC视频属性的抓取
BV号为1RT4y157vC的视频属性抓取结束
开始BV号为1gh411o7bU视频属性的抓取
BV号为1gh411o7bU的视频属性抓取结束
开始BV号为1S54y1e75c视频属性的抓取
BV号为1S54y1e75c的视频属性抓取结束
开始BV号为1h54y1e7MC视频属性的抓取
BV号为1h54y1e7MC的视频属性抓取结束
开始BV号为1fV411U7tz视频属性的抓取
BV号为1fV411U7tz的视频属性抓取结束
开始BV号为1gA411n78a视频属性的抓取
BV号为1gA411n78a的视频属性抓取结束
开始BV号为1yf4y1Q79H视频属性的抓取
BV号为1yf4y1Q79H的视频属性抓取结束
开始BV号为1sC4y187pE视频属性的抓取
BV号为1sC4y187pE的视频属性抓取结束
开始BV号为1nC4y1t7ZU视频属性的抓取
BV号为1nC4y1t7ZU的视频属性抓取结束
开始BV号为1wi4y1g7Jw视频属性的抓取
BV号为1wi4y1g7Jw的视频属性抓取结束
开始BV号为12i4y1u7Fj视频属性的抓取
BV号为12i4y1u7Fj的视频属性抓取结束
开始BV号为1d5411a7Fy视频属性的抓取
BV号为1d5411a7Fy的视频属性抓取结束
开始BV号为15A411n7po视频属性的抓取
BV号为15A411n7po的视频属性抓取结束
开始BV号为1ka4y1E7QR视频属性的抓取
BV号为1ka4y1E7QR的视频属性抓取结束
开始BV号为1Kv41117yt视频属性的抓取
BV号为1Kv41117yt的视频属性抓取结束
开始BV号为1oK4y1e7Rn视频属性的抓取
BV号为1oK4y1e7Rn的视频属性抓取结束
开始BV号为1qT4y1572x视频属性的抓取
BV号为1qT4y1572x的视频属性抓取结束
开始BV号为1Q5411a7JL视频属性的抓取
BV号为1Q5411a7JL的视

开始BV号为1hZ4y1N7WS视频属性的抓取
BV号为1hZ4y1N7WS的视频属性抓取结束
开始BV号为1F54y1m74G视频属性的抓取
BV号为1F54y1m74G的视频属性抓取结束
开始BV号为1Wa4y1j7cK视频属性的抓取
BV号为1Wa4y1j7cK的视频属性抓取结束
开始BV号为1sa4y1j7oJ视频属性的抓取
BV号为1sa4y1j7oJ的视频属性抓取结束
开始BV号为17K411K7q3视频属性的抓取
BV号为17K411K7q3的视频属性抓取结束
开始BV号为1w541187c3视频属性的抓取
BV号为1w541187c3的视频属性抓取结束
开始BV号为19Z4y1P7n1视频属性的抓取
BV号为19Z4y1P7n1的视频属性抓取结束
开始BV号为1Az4y1Z7Qq视频属性的抓取
BV号为1Az4y1Z7Qq的视频属性抓取结束
开始BV号为1pK411P71Z视频属性的抓取
BV号为1pK411P71Z的视频属性抓取结束
开始BV号为1FA411H7vK视频属性的抓取
BV号为1FA411H7vK的视频属性抓取结束
开始BV号为14h411X7A5视频属性的抓取
BV号为14h411X7A5的视频属性抓取结束
开始BV号为1kt4y1i7Kc视频属性的抓取
BV号为1kt4y1i7Kc的视频属性抓取结束
开始BV号为1bp4y1e7hb视频属性的抓取
BV号为1bp4y1e7hb的视频属性抓取结束
开始BV号为1wz4y1f7Hj视频属性的抓取
BV号为1wz4y1f7Hj的视频属性抓取结束
开始BV号为13K4y1Y7fn视频属性的抓取
BV号为13K4y1Y7fn的视频属性抓取结束
开始BV号为1s54y1m71L视频属性的抓取
BV号为1s54y1m71L的视频属性抓取结束
开始BV号为1jv411y7t4视频属性的抓取
BV号为1jv411y7t4的视频属性抓取结束
开始BV号为1Bk4y1y7v3视频属性的抓取
BV号为1Bk4y1y7v3的视频属性抓取结束
开始BV号为1zK411P75R视频属性的抓取
BV号为1zK411P75R的视频属性抓取结束
开始BV号为14y4y1y7qy视频属性的抓取
BV号为14y4y1y7qy的视频属性抓取结束
开始BV号为17i4y1j7RY视频属性的抓取
BV号为17i4y1j7RY的视

In [10]:
driver.switch_to_window(driver.window_handles[-1])

<ipython-input-10-dd0a1399d54d>:1: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


In [12]:
channel_shenghuo_shougong.groupby('时间').agg('count')

,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,自制or转载,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
时间,,,,,,,,,,,,,,,,,,,,
10月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
1月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
2月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
3月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
4月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
5月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
6月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
7月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
8月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99


In [13]:
channel_shenghuo_shougong.to_json('channel_shenghuo_shougong.json')